In [1]:
import pandas as pd
import numpy as np
import os 

import tensorflow as tf
from tensorflow import keras
strategy = tf.distribute.OneDeviceStrategy(device='/gpu:0')

!wget https://storage.googleapis.com/cloud-tpu-checkpoints/bert/keras_bert/multi_cased_L-12_H-768_A-12.tar.gz
!tar xvzf multi_cased_L-12_H-768_A-12.tar.gz

!pip install -q tf-models-official==2.3.0
from official.nlp.albert import configs as albert_configs
from official.nlp.bert import configs
from official.modeling import tf_utils
from official.nlp.modeling import networks

from sklearn.model_selection import KFold
from sklearn import metrics

--2021-04-22 18:40:54--  https://storage.googleapis.com/cloud-tpu-checkpoints/bert/keras_bert/multi_cased_L-12_H-768_A-12.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.13.240, 172.217.15.80, 172.217.7.176, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.13.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 660200694 (630M) [application/octet-stream]
Saving to: ‘multi_cased_L-12_H-768_A-12.tar.gz’

multi_cased_L-12_H- 100%[===================>] 629.62M  56.1MB/s    in 11s     

2021-04-22 18:41:05 (57.1 MB/s) - ‘multi_cased_L-12_H-768_A-12.tar.gz’ saved [660200694/660200694]

./multi_cased_L-12_H-768_A-12/
./multi_cased_L-12_H-768_A-12/vocab.txt
./multi_cased_L-12_H-768_A-12/bert_model.ckpt.index
./multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
./multi_cased_L-12_H-768_A-12/bert_config.json
     |████████████████████████████████| 849kB 19.5MB/s 
     |████████████████████████████████| 174

In [2]:
#2
MODEL_DIR = "multi_cased_L-12_H-768_A-12" #@param {type:"string"} ['multi_cased_L-12_H-768_A-12']
MLP_DIMS = "" #@param {type:"string"}
MAX_SEQ_LENGTH = 64 #@param {type:"integer"}
N_CLASSES = 3 #@param {type:"integer"}

if MLP_DIMS:
  mlp_dims = MLP_DIMS.split(",")
  mlp_dims = [ int(x) for x in mlp_dims ]
  print(mlp_dims)
else:
  mlp_dims = []

config_file_path = "/content/{}/bert_config.json".format(MODEL_DIR)
bert_config = configs.BertConfig.from_json_file(config_file_path)

#se flag == 1 então path de BERT e checkpoint.restore
#se flag == 2 então path de BERT e sem checkpoint.restore
#se flag == 3 entao path nossos checkpoint
def create_model(flag):
  if flag == 3:
    checkpoint_path = "training_1/cp.ckpt"
  else:
    checkpoint_path = "/content/{}/bert_model.ckpt".format(MODEL_DIR)

  cfg = bert_config
  print(cfg)

  kwargs = dict(
        vocab_size=cfg.vocab_size,
        hidden_size=cfg.hidden_size,
        num_layers=cfg.num_hidden_layers,
        num_attention_heads=cfg.num_attention_heads,
        intermediate_size=cfg.intermediate_size,
        activation=tf_utils.get_activation(cfg.hidden_act),
        dropout_rate=cfg.hidden_dropout_prob,
        attention_dropout_rate=cfg.attention_probs_dropout_prob,
        sequence_length=MAX_SEQ_LENGTH,
        max_sequence_length=cfg.max_position_embeddings,
        type_vocab_size=cfg.type_vocab_size,
        embedding_width=cfg.embedding_size,
        initializer=tf.keras.initializers.TruncatedNormal(
            stddev=cfg.initializer_range))

  bert_encoder = networks.TransformerEncoder(**kwargs)

  checkpoint = tf.train.Checkpoint(model=bert_encoder)
  options = tf.train.CheckpointOptions(experimental_io_device="/job:localhost")
  if flag == 1:
    checkpoint.restore(checkpoint_path, options=options).assert_consumed().run_restore_ops()
  
  word_ids_input = tf.keras.layers.Input(
      shape=(MAX_SEQ_LENGTH,), dtype=tf.int32, name='word_ids')
  mask_input = tf.keras.layers.Input(
      shape=(MAX_SEQ_LENGTH,), dtype=tf.int32, name='masks')
  type_ids_input = tf.keras.layers.Input(
      shape=(MAX_SEQ_LENGTH,), dtype=tf.int32, name='type_ids')

  sequence_output, pooled_output = bert_encoder(
        [word_ids_input, mask_input, type_ids_input])
  
  print(sequence_output)
  print(pooled_output)
  
  # Isto só é necessário se não se quiser usar fine-tuning (para ser mais rapido)
  #pooled_output = tf.stop_gradient(pooled_output)

  if mlp_dims:
    mlp = tf.keras.layers.Dense(mlp_dims[0], activation='tanh')(pooled_output)
    for dims in mlp_dims[1:]:
      mlp = tf.keras.layers.Dense(dims, activation='tanh')(mlp)
  else:
    mlp = pooled_output

  output = tf.keras.layers.Dense(N_CLASSES, activation='softmax')(mlp)

  model = tf.keras.Model(
      inputs=[word_ids_input, mask_input, type_ids_input], outputs=output)
  return model

In [3]:
#3
MODEL_DIR = "multi_cased_L-12_H-768_A-12" #@param {type:"string"} ['multi_cased_L-12_H-768_A-12']

from official.nlp.bert import tokenization

vocab_file = "/content/{}/vocab.txt".format(MODEL_DIR)

tokenizer = tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=False)

#tokenizer.tokenize("Este é um exemplo de utilização de um BERT tokenizer em português.")

In [4]:
#4
def transform_to_dataset(input_text, labels):
  def generator():
    for i, (key, text) in enumerate(input_text.iteritems()):
      label = labels[key]
      unicode_text = tokenization.convert_to_unicode(text)
      tokens = tokenizer.tokenize(unicode_text)
      tokens = ['[CLS]'] + tokens + ['[SEP]']
      token_ids = tokenizer.convert_tokens_to_ids(tokens)

      mask_ids = []
      type_ids = []
      for i in range(len(token_ids)):
        mask_ids.append(1)
        type_ids.append(0)

      pad_id = tokenizer.convert_tokens_to_ids(['[PAD]'])[0]

      if len(token_ids) > MAX_SEQ_LENGTH:
        token_ids = token_ids[:MAX_SEQ_LENGTH]
        tokens = tokens[:MAX_SEQ_LENGTH]
        mask_ids = mask_ids[:MAX_SEQ_LENGTH]
        type_ids = type_ids[:MAX_SEQ_LENGTH]
      else:
        for _ in range(MAX_SEQ_LENGTH - len(token_ids)):
          token_ids.append(pad_id)
          tokens.append('[PAD]')
          mask_ids.append(0)
          type_ids.append(1)

      yield {"word_ids": token_ids,
             "masks": mask_ids, "type_ids": type_ids}, label
  return generator

In [5]:
df = pd.read_csv("https://raw.githubusercontent.com/b2wdigital/b2w-reviews01/master/B2W-Reviews01.csv", sep = ";")

#New feature "label" - 0:'Negativo, 1:'Neutro' and 2:'Positivo' 
df['label'] = np.where(df['overall_rating'] >= 4, 2, np.where(df['overall_rating'] <= 2, 0, 1))

df_Anotados = df[:800]
df_2018 = df[800:10877]
df_naoAnotados = df[10877:]


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
#6 1 - Pré-treino: Treinar para uma tarefa que não é bem o que querem mas para a qual conseguem gerar muitos dados.
#- Tirar todos os dados que tinham anotado à mão do conjunto dos 4 semestres.
#- Treinar o BERT com estes dados baseado nas notas do professor.
#- Podem decidir se querem separar 20% para validação ou não mas não é obrigatório.

input_text_train = df_2018['review_text']
label_train = df_2018['label']

train_dataset_na = tf.data.Dataset.from_generator(
    transform_to_dataset(input_text_train, label_train),
    output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
    output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))

print(train_dataset_na)
train_dataset_na = train_dataset_na.batch(32)

<FlatMapDataset shapes: ({word_ids: (64,), masks: (64,), type_ids: (64,)}, ()), types: ({word_ids: tf.int64, masks: tf.int64, type_ids: tf.int64}, tf.int64)>


In [7]:
#7
STEPS_PER_EPOCH = -1 #@param {type:"integer"}
VALIDATION_STEPS = -1 #@param {type:"integer"}

steps_per_epoch = STEPS_PER_EPOCH
if STEPS_PER_EPOCH <= 0:
  steps_per_epoch = None

validation_steps = VALIDATION_STEPS
if VALIDATION_STEPS <= 0:
  validation_steps = None

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

with strategy.scope():
  #model = tf.keras.models.clone_model(local_model)
  model = create_model(1)
  model.summary()
  # Dica: o learning rate com o BERT tem sempre de ser muito pequeno, caso contrário, ele vai esquecer o pre-treino durante os primeiros passos.
  opt = keras.optimizers.Adam(learning_rate=0.00001)
  model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
  history = model.fit(
        train_dataset_na,
        epochs=10, steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps, callbacks=[cp_callback])

KerasTensor(type_spec=TensorSpec(shape=(None, 64, 768), dtype=tf.float32, name=None), name='transformer_encoder/transformer/layer_11/output_layer_norm/batchnorm/add_1:0', description="created by layer 'transformer_encoder'")
KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='transformer_encoder/pooler_transform/Tanh:0', description="created by layer 'transformer_encoder'")
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word_ids (InputLayer)           [(None, 64)]         0                                            
__________________________________________________________________________________________________
masks (InputLayer)              [(None, 64)]         0                                            
_____________________________________________________________________________

In [8]:
#8 
#- Usar apenas os dados que tinham anotado à mão.

df_train_a = df_Anotados[:640]
df_test_a = df_Anotados[640:]

input_text_train = df_train_a['review_text']
label_train = df_train_a['label']
input_text_test = df_test_a['review_text']
label_test = df_test_a['label']

# Merge inputs and targets
inputs = pd.Series(np.concatenate((input_text_train, input_text_test), axis=0))
targets = pd.Series(np.concatenate((label_train, label_test), axis=0))

# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Define per-fold score containers for from-scratch
acc_per_fold_640_fs = []
loss_per_fold_640_fs = []
precision_per_fold_640_fs = []
recall_per_fold_640_fs = []

acc_per_fold_320_fs = []
loss_per_fold_320_fs = []
precision_per_fold_320_fs = []
recall_per_fold_320_fs = []

acc_per_fold_160_fs = []
loss_per_fold_160_fs = []
precision_per_fold_160_fs = []
recall_per_fold_160_fs = []

acc_per_fold_80_fs = []
loss_per_fold_80_fs = []
precision_per_fold_80_fs = []
recall_per_fold_80_fs = []

acc_per_fold_40_fs = []
loss_per_fold_40_fs = []
precision_per_fold_40_fs = []
recall_per_fold_40_fs = []

f1_fold_640_fs = []
f1_fold_320_fs = []
f1_fold_160_fs = []
f1_fold_80_fs = []
f1_fold_40_fs = []


# Define per-fold score containers for fine-tuned
acc_per_fold_640_ft = []
loss_per_fold_640_ft = []
precision_per_fold_640_ft = []
recall_per_fold_640_ft = []

acc_per_fold_320_ft = []
loss_per_fold_320_ft = []
precision_per_fold_320_ft = []
recall_per_fold_320_ft = []

acc_per_fold_160_ft = []
loss_per_fold_160_ft = []
precision_per_fold_160_ft = []
recall_per_fold_160_ft = []

acc_per_fold_80_ft = []
loss_per_fold_80_ft = []
precision_per_fold_80_ft = []
recall_per_fold_80_ft = []

acc_per_fold_40_ft = []
loss_per_fold_40_ft = []
precision_per_fold_40_ft = []
recall_per_fold_40_ft = []

f1_fold_640_ft = []
f1_fold_320_ft = []
f1_fold_160_ft = []
f1_fold_80_ft = []
f1_fold_40_ft = []

In [10]:
#9
# - Zero-shot: Carregar o checkpoint to pré-treino avaliar directamente nos 20%

STEPS_PER_EPOCH = -1 #@param {type:"integer"}
VALIDATION_STEPS = -1 #@param {type:"integer"}

steps_per_epoch = STEPS_PER_EPOCH
if STEPS_PER_EPOCH <= 0:
  steps_per_epoch = None

validation_steps = VALIDATION_STEPS
if VALIDATION_STEPS <= 0:
  validation_steps = None

# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []
precision_per_fold = []
recall_per_fold = []

with strategy.scope():
  # K-fold Cross Validation model evaluation
  fold_no = 1
  for train, test in kfold.split(inputs, targets):
    test_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[test], targets[test]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))

    test_dataset = test_dataset.batch(32)
    #model = tf.keras.models.clone_model(local_model)
    model = create_model(3)
    checkpoint_path = "training_1/cp.ckpt"
    model.load_weights(checkpoint_path)
    model.summary()
    # Dica: o learning rate com o BERT tem sempre de ser muito pequeno, caso contrário, ele vai esquecer o pre-treino durante os primeiros passos.
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
    history_zeroShot = model.evaluate(
        test_dataset,
        batch_size=10, steps=10)
    
    scores = model.evaluate(test_dataset, verbose=0)
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    predictions = model.predict(test_dataset)
    matrix = metrics.confusion_matrix(targets[test],predictions.argmax(1))
    print(matrix)
    recall_per_fold.append(np.mean(np.diag(matrix) / np.sum(matrix, axis = 1))*100)
    precision_per_fold.append(np.mean(np.diag(matrix) / np.sum(matrix, axis = 0))*100)

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}% - Precision: {precision_per_fold[i]} - Recall: {recall_per_fold[i]}')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print(f'> Precision: {np.mean(precision_per_fold)}(+- {np.std(precision_per_fold)})')
print(f'> Recall: {np.mean(recall_per_fold)}(+- {np.std(recall_per_fold)})')
print('------------------------------------------------------------------------') 

KerasTensor(type_spec=TensorSpec(shape=(None, 64, 768), dtype=tf.float32, name=None), name='transformer_encoder_1/transformer/layer_11/output_layer_norm/batchnorm/add_1:0', description="created by layer 'transformer_encoder_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='transformer_encoder_1/pooler_transform/Tanh:0', description="created by layer 'transformer_encoder_1'")
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word_ids (InputLayer)           [(None, 64)]         0                                            
__________________________________________________________________________________________________
masks (InputLayer)              [(None, 64)]         0                                            
___________________________________________________________________

In [7]:
#10
# De raíz: Não carregar o checkpoint to pré-treino e treinar tudo

STEPS_PER_EPOCH = -1 #@param {type:"integer"}
VALIDATION_STEPS = -1 #@param {type:"integer"}

steps_per_epoch = STEPS_PER_EPOCH
if STEPS_PER_EPOCH <= 0:
  steps_per_epoch = None

validation_steps = VALIDATION_STEPS
if VALIDATION_STEPS <= 0:
  validation_steps = None

with strategy.scope():
  # K-fold Cross Validation model evaluation
  fold_no = 1
  for train, test in kfold.split(inputs, targets):
    test_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[test], targets[test]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    test_dataset = test_dataset.batch(32)

    #640
    train_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[train], targets[train]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    train_dataset = train_dataset.batch(32)

    #model = tf.keras.models.clone_model(local_model)
    model = create_model(1)
    model.summary()
    # Dica: o learning rate com o BERT tem sempre de ser muito pequeno, caso contrário, ele vai esquecer o pre-treino durante os primeiros passos.
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
    history_root = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=10, steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps)
    
    scores = model.evaluate(test_dataset, verbose=0)
    acc_per_fold_640_fs.append(scores[1] * 100)
    loss_per_fold_640_fs.append(scores[0])

    predictions = model.predict(test_dataset)
    matrix = metrics.confusion_matrix(targets[test],predictions.argmax(1))
    p = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 0))*100)
    r = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 1))*100)
    recall_per_fold_640_fs.append(r)
    precision_per_fold_640_fs.append(p)
    f1_fold_640_fs.append(2*((p*r)/(p+r)))

    #320
    train_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[train[:320]], targets[train[:320]]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    train_dataset = train_dataset.batch(32)

    #model = tf.keras.models.clone_model(local_model)
    model = create_model(1)
    model.summary()
    # Dica: o learning rate com o BERT tem sempre de ser muito pequeno, caso contrário, ele vai esquecer o pre-treino durante os primeiros passos.
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
    history_root = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=10, steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps)
    
    scores = model.evaluate(test_dataset, verbose=0)
    acc_per_fold_320_fs.append(scores[1] * 100)
    loss_per_fold_320_fs.append(scores[0])

    predictions = model.predict(test_dataset)
    matrix = metrics.confusion_matrix(targets[test],predictions.argmax(1))
    p = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 0))*100)
    r = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 1))*100)
    recall_per_fold_320_fs.append(r)
    precision_per_fold_320_fs.append(p)
    f1_fold_320_fs.append(2*((p*r)/(p+r)))

    #160
    train_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[train[:160]], targets[train[:160]]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    train_dataset = train_dataset.batch(32)

    #model = tf.keras.models.clone_model(local_model)
    model = create_model(1)
    model.summary()
    # Dica: o learning rate com o BERT tem sempre de ser muito pequeno, caso contrário, ele vai esquecer o pre-treino durante os primeiros passos.
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
    history_root = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=10, steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps)
    
    scores = model.evaluate(test_dataset, verbose=0)
    acc_per_fold_160_fs.append(scores[1] * 100)
    loss_per_fold_160_fs.append(scores[0])

    predictions = model.predict(test_dataset)
    matrix = metrics.confusion_matrix(targets[test],predictions.argmax(1))
    p = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 0))*100)
    r = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 1))*100)
    recall_per_fold_160_fs.append(r)
    precision_per_fold_160_fs.append(p)
    f1_fold_160_fs.append(2*((p*r)/(p+r)))
    
    #80
    train_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[train[:80]], targets[train[:80]]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    train_dataset = train_dataset.batch(32)

    #model = tf.keras.models.clone_model(local_model)
    model = create_model(1)
    model.summary()
    # Dica: o learning rate com o BERT tem sempre de ser muito pequeno, caso contrário, ele vai esquecer o pre-treino durante os primeiros passos.
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
    history_root = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=10, steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps)
    
    scores = model.evaluate(test_dataset, verbose=0)
    acc_per_fold_80_fs.append(scores[1] * 100)
    loss_per_fold_80_fs.append(scores[0])

    predictions = model.predict(test_dataset)
    matrix = metrics.confusion_matrix(targets[test],predictions.argmax(1))
    p = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 0))*100)
    r = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 1))*100)
    recall_per_fold_80_fs.append(r)
    precision_per_fold_80_fs.append(p)
    f1_fold_80_fs.append(2*((p*r)/(p+r)))

    #40
    train_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[train[:40]], targets[train[:40]]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    train_dataset = train_dataset.batch(32)

    #model = tf.keras.models.clone_model(local_model)
    model = create_model(1)
    model.summary()
    # Dica: o learning rate com o BERT tem sempre de ser muito pequeno, caso contrário, ele vai esquecer o pre-treino durante os primeiros passos.
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
    history_root = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=10, steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps)
    
    scores = model.evaluate(test_dataset, verbose=0)
    acc_per_fold_40_fs.append(scores[1] * 100)
    loss_per_fold_40_fs.append(scores[0])

    predictions = model.predict(test_dataset)
    matrix = metrics.confusion_matrix(targets[test],predictions.argmax(1))
    p = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 0))*100)
    r = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 1))*100)
    recall_per_fold_40_fs.append(r)
    precision_per_fold_40_fs.append(p)
    f1_fold_40_fs.append(2*((p*r)/(p+r)))

KerasTensor(type_spec=TensorSpec(shape=(None, 64, 768), dtype=tf.float32, name=None), name='transformer_encoder/transformer/layer_11/output_layer_norm/batchnorm/add_1:0', description="created by layer 'transformer_encoder'")
KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='transformer_encoder/pooler_transform/Tanh:0', description="created by layer 'transformer_encoder'")
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word_ids (InputLayer)           [(None, 64)]         0                                            
__________________________________________________________________________________________________
masks (InputLayer)              [(None, 64)]         0                                            
_____________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:165: RuntimeWarning: invalid value encountered in true_divide


KerasTensor(type_spec=TensorSpec(shape=(None, 64, 768), dtype=tf.float32, name=None), name='transformer_encoder_4/transformer/layer_11/output_layer_norm/batchnorm/add_1:0', description="created by layer 'transformer_encoder_4'")
KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='transformer_encoder_4/pooler_transform/Tanh:0', description="created by layer 'transformer_encoder_4'")
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word_ids (InputLayer)           [(None, 64)]         0                                            
__________________________________________________________________________________________________
masks (InputLayer)              [(None, 64)]         0                                            
___________________________________________________________________

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:201: RuntimeWarning: invalid value encountered in true_divide


KerasTensor(type_spec=TensorSpec(shape=(None, 64, 768), dtype=tf.float32, name=None), name='transformer_encoder_5/transformer/layer_11/output_layer_norm/batchnorm/add_1:0', description="created by layer 'transformer_encoder_5'")
KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='transformer_encoder_5/pooler_transform/Tanh:0', description="created by layer 'transformer_encoder_5'")
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word_ids (InputLayer)           [(None, 64)]         0                                            
__________________________________________________________________________________________________
masks (InputLayer)              [(None, 64)]         0                                            
___________________________________________________________________

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:129: RuntimeWarning: invalid value encountered in true_divide


KerasTensor(type_spec=TensorSpec(shape=(None, 64, 768), dtype=tf.float32, name=None), name='transformer_encoder_23/transformer/layer_11/output_layer_norm/batchnorm/add_1:0', description="created by layer 'transformer_encoder_23'")
KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='transformer_encoder_23/pooler_transform/Tanh:0', description="created by layer 'transformer_encoder_23'")
Model: "model_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word_ids (InputLayer)           [(None, 64)]         0                                            
__________________________________________________________________________________________________
masks (InputLayer)              [(None, 64)]         0                                            
______________________________________________________________

In [8]:
# == Provide average scores ==
print('-------------------------------640-----------------------------------------')
#print('Score per fold')
#for i in range(0, len(acc_per_fold_640_fs)):
#  print('------------------------------------------------------------------------')
#  print(f'> Fold {i+1} - Loss: {loss_per_fold_640_fs[i]} - Accuracy: {acc_per_fold_640_fs[i]}% - Precision: {precision_per_fold_640_fs[i]} - Recall: {recall_per_fold_640_fs[i]}')
#print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_640_fs)} (+- {np.std(acc_per_fold_640_fs)})')
print(f'> Loss: {np.mean(loss_per_fold_640_fs)}')
print(f'> F1: {np.mean(f1_fold_640_fs)}(+- {np.std(f1_fold_640_fs)})')
print(f'> Precision: {np.mean(precision_per_fold_640_fs)}(+- {np.std(precision_per_fold_640_fs)})')
print(f'> Recall: {np.mean(recall_per_fold_640_fs)}(+- {np.std(recall_per_fold_640_fs)})')
print('------------------------------------------------------------------------') 

# == Provide average scores ==
print('-------------------------------320-----------------------------------------')
#print('Score per fold')
#for i in range(0, len(acc_per_fold_320_fs)):
#  print('------------------------------------------------------------------------')
#  print(f'> Fold {i+1} - Loss: {loss_per_fold_320_fs[i]} - Accuracy: {acc_per_fold_320_fs[i]}% - Precision: {precision_per_fold_320_fs[i]} - Recall: {recall_per_fold_320_fs[i]}')
#print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_320_fs)} (+- {np.std(acc_per_fold_320_fs)})')
print(f'> Loss: {np.mean(loss_per_fold_320_fs)}')
print(f'> F1: {np.mean(f1_fold_320_fs)}(+- {np.std(f1_fold_320_fs)})')
print(f'> Precision: {np.mean(precision_per_fold_320_fs)}(+- {np.std(precision_per_fold_320_fs)})')
print(f'> Recall: {np.mean(recall_per_fold_320_fs)}(+- {np.std(recall_per_fold_320_fs)})')
print('------------------------------------------------------------------------') 

# == Provide average scores ==
print('-------------------------------160-----------------------------------------')
#print('Score per fold')
#for i in range(0, len(acc_per_fold_160_fs)):
#  print('------------------------------------------------------------------------')
#  print(f'> Fold {i+1} - Loss: {loss_per_fold_160_fs[i]} - Accuracy: {acc_per_fold_160_fs[i]}% - Precision: {precision_per_fold_160_fs[i]} - Recall: {recall_per_fold_160_fs[i]}')
#print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_160_fs)} (+- {np.std(acc_per_fold_160_fs)})')
print(f'> Loss: {np.mean(loss_per_fold_160_fs)}')
print(f'> F1: {np.mean(f1_fold_160_fs)}(+- {np.std(f1_fold_160_fs)})')
print(f'> Precision: {np.mean(precision_per_fold_160_fs)}(+- {np.std(precision_per_fold_160_fs)})')
print(f'> Recall: {np.mean(recall_per_fold_160_fs)}(+- {np.std(recall_per_fold_160_fs)})')
print('------------------------------------------------------------------------') 

# == Provide average scores ==
print('-------------------------------80-----------------------------------------')
#print('Score per fold')
#for i in range(0, len(acc_per_fold_80_fs)):
#  print('------------------------------------------------------------------------')
#  print(f'> Fold {i+1} - Loss: {loss_per_fold_80_fs[i]} - Accuracy: {acc_per_fold_80_fs[i]}% - Precision: {np.nan_to_num(precision_per_fold_80_fs[i])} - Recall: {recall_per_fold_80_fs[i]}')
#print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_80_fs)} (+- {np.std(acc_per_fold_80_fs)})')
print(f'> Loss: {np.mean(loss_per_fold_80_fs)}')
print(f'> F1: {np.mean(f1_fold_80_fs)}(+- {np.std(f1_fold_80_fs)})')
print(f'> Precision: {np.mean(np.nan_to_num(precision_per_fold_80_fs))}(+- {np.std(np.nan_to_num(precision_per_fold_80_fs))})')
print(f'> Recall: {np.mean(recall_per_fold_80_fs)}(+- {np.std(recall_per_fold_80_fs)})')
print('------------------------------------------------------------------------') 

# == Provide average scores ==
print('-------------------------------40-----------------------------------------')
#print('Score per fold')
#for i in range(0, len(acc_per_fold_40_fs)):
#  print('------------------------------------------------------------------------')
#  print(f'> Fold {i+1} - Loss: {loss_per_fold_40_fs[i]} - Accuracy: {acc_per_fold_40_fs[i]}% - Precision: {np.nan_to_num(precision_per_fold_40_fs[i])} - Recall: {recall_per_fold_40_fs[i]}')
#print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_40_fs)} (+- {np.std(acc_per_fold_40_fs)})')
print(f'> Loss: {np.mean(loss_per_fold_40_fs)}')
print(f'> F1: {np.mean(f1_fold_40_fs)}(+- {np.std(f1_fold_40_fs)})')
print(f'> Precision: {np.mean(np.nan_to_num(precision_per_fold_40_fs))}(+- {np.std(np.nan_to_num(precision_per_fold_40_fs))})')
print(f'> Recall: {np.mean(recall_per_fold_40_fs)}(+- {np.std(recall_per_fold_40_fs)})')
print('------------------------------------------------------------------------') 

-------------------------------640-----------------------------------------
Average scores for all folds:
> Accuracy: 72.74999976158142 (+- 8.629525852709971)
> Loss: 1.0263736724853516
> F1: 56.97039149959592(+- 2.52892940269094)
> Precision: 55.536276668192556(+- 3.110882420322247)
> Recall: 58.519153073228935(+- 2.1910300547715456)
------------------------------------------------------------------------
-------------------------------320-----------------------------------------
Average scores for all folds:
> Accuracy: 73.49999904632568 (+- 3.3213337034679036)
> Loss: 1.0563324689865112
> F1: 54.02146516585992(+- 3.3820367479820947)
> Precision: 58.68198085076047(+- 11.366267296111289)
> Recall: 51.544036170473376(+- 4.336620063473227)
------------------------------------------------------------------------
-------------------------------160-----------------------------------------
Average scores for all folds:
> Accuracy: 71.875 (+- 2.2707382249318275)
> Loss: 0.8623051285743714
> 

In [9]:
#11
#- Fine-tuned: Carregar o checkpoint do pré-treino e treinar durante o mesmo número epochs que a variante de raíz

STEPS_PER_EPOCH = -1 #@param {type:"integer"}
VALIDATION_STEPS = -1 #@param {type:"integer"}

steps_per_epoch = STEPS_PER_EPOCH
if STEPS_PER_EPOCH <= 0:
  steps_per_epoch = None

validation_steps = VALIDATION_STEPS
if VALIDATION_STEPS <= 0:
  validation_steps = None

with strategy.scope():
  # K-fold Cross Validation model evaluation
  fold_no = 1
  for train, test in kfold.split(inputs, targets):
    test_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[test], targets[test]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    test_dataset = test_dataset.batch(32)

    #640
    train_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[train], targets[train]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    
    train_dataset = train_dataset.batch(32)

    #model = tf.keras.models.clone_model(local_model)
    model = create_model(3)
    checkpoint_path = "training_1/cp.ckpt"
    model.load_weights(checkpoint_path)
    model.summary()
    # Dica: o learning rate com o BERT tem sempre de ser muito pequeno, caso contrário, ele vai esquecer o pre-treino durante os primeiros passos.
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
    history_fineTuned = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=10, steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps)

    scores = model.evaluate(test_dataset, verbose=0)
    acc_per_fold_640_ft.append(scores[1] * 100)
    loss_per_fold_640_ft.append(scores[0])

    predictions = model.predict(test_dataset)
    matrix = metrics.confusion_matrix(targets[test],predictions.argmax(1))
    p = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 0))*100)
    r = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 1))*100)
    recall_per_fold_640_ft.append(r)
    precision_per_fold_640_ft.append(p)
    f1_fold_640_ft.append(2*((p*r)/(p+r)))

    #320
    train_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[train[:320]], targets[train[:320]]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    
    train_dataset = train_dataset.batch(32)

    #model = tf.keras.models.clone_model(local_model)
    model = create_model(3)
    checkpoint_path = "training_1/cp.ckpt"
    model.load_weights(checkpoint_path)
    model.summary()
    # Dica: o learning rate com o BERT tem sempre de ser muito pequeno, caso contrário, ele vai esquecer o pre-treino durante os primeiros passos.
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
    history_fineTuned = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=10, steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps)

    scores = model.evaluate(test_dataset, verbose=0)
    acc_per_fold_320_ft.append(scores[1] * 100)
    loss_per_fold_320_ft.append(scores[0])

    predictions = model.predict(test_dataset)
    matrix = metrics.confusion_matrix(targets[test],predictions.argmax(1))
    p = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 0))*100)
    r = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 1))*100)
    recall_per_fold_320_ft.append(r)
    precision_per_fold_320_ft.append(p)
    f1_fold_320_ft.append(2*((p*r)/(p+r)))

    #160
    train_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[train[:160]], targets[train[:160]]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    
    train_dataset = train_dataset.batch(32)

    #model = tf.keras.models.clone_model(local_model)
    model = create_model(3)
    checkpoint_path = "training_1/cp.ckpt"
    model.load_weights(checkpoint_path)
    model.summary()
    # Dica: o learning rate com o BERT tem sempre de ser muito pequeno, caso contrário, ele vai esquecer o pre-treino durante os primeiros passos.
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
    history_fineTuned = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=10, steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps)

    scores = model.evaluate(test_dataset, verbose=0)
    acc_per_fold_160_ft.append(scores[1] * 100)
    loss_per_fold_160_ft.append(scores[0])

    predictions = model.predict(test_dataset)
    matrix = metrics.confusion_matrix(targets[test],predictions.argmax(1))
    p = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 0))*100)
    r = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 1))*100)
    recall_per_fold_160_ft.append(r)
    precision_per_fold_160_ft.append(p)
    f1_fold_160_ft.append(2*((p*r)/(p+r)))

    #80
    train_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[train[:80]], targets[train[:80]]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    
    train_dataset = train_dataset.batch(32)

    #model = tf.keras.models.clone_model(local_model)
    model = create_model(3)
    checkpoint_path = "training_1/cp.ckpt"
    model.load_weights(checkpoint_path)
    model.summary()
    # Dica: o learning rate com o BERT tem sempre de ser muito pequeno, caso contrário, ele vai esquecer o pre-treino durante os primeiros passos.
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
    history_fineTuned = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=10, steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps)

    scores = model.evaluate(test_dataset, verbose=0)
    acc_per_fold_80_ft.append(scores[1] * 100)
    loss_per_fold_80_ft.append(scores[0])

    predictions = model.predict(test_dataset)
    matrix = metrics.confusion_matrix(targets[test],predictions.argmax(1))
    p = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 0))*100)
    r = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 1))*100)
    recall_per_fold_80_ft.append(r)
    precision_per_fold_80_ft.append(p)
    f1_fold_80_ft.append(2*((p*r)/(p+r)))

    #40
    train_dataset = tf.data.Dataset.from_generator(
      transform_to_dataset(inputs[train[:40]], targets[train[:40]]),
      output_types=({"word_ids": tf.int64,
                   "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
      output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
    
    train_dataset = train_dataset.batch(32)

    #model = tf.keras.models.clone_model(local_model)
    model = create_model(3)
    checkpoint_path = "training_1/cp.ckpt"
    model.load_weights(checkpoint_path)
    model.summary()
    # Dica: o learning rate com o BERT tem sempre de ser muito pequeno, caso contrário, ele vai esquecer o pre-treino durante os primeiros passos.
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])
    history_fineTuned = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=10, steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps)

    scores = model.evaluate(test_dataset, verbose=0)
    acc_per_fold_40_ft.append(scores[1] * 100)
    loss_per_fold_40_ft.append(scores[0])

    predictions = model.predict(test_dataset)
    matrix = metrics.confusion_matrix(targets[test],predictions.argmax(1))
    p = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 0))*100)
    r = np.nan_to_num(np.mean(np.diag(matrix) / np.sum(matrix, axis = 1))*100)
    recall_per_fold_40_ft.append(r)
    precision_per_fold_40_ft.append(p)
    f1_fold_40_ft.append(2*((p*r)/(p+r)))


KerasTensor(type_spec=TensorSpec(shape=(None, 64, 768), dtype=tf.float32, name=None), name='transformer_encoder_1/transformer/layer_11/output_layer_norm/batchnorm/add_1:0', description="created by layer 'transformer_encoder_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 768), dtype=tf.float32, name=None), name='transformer_encoder_1/pooler_transform/Tanh:0', description="created by layer 'transformer_encoder_1'")
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word_ids (InputLayer)           [(None, 64)]         0                                            
__________________________________________________________________________________________________
masks (InputLayer)              [(None, 64)]         0                                            
___________________________________________________________________

In [10]:
# == Provide average scores ==
print('-------------------------------640-----------------------------------------')
#print('Score per fold')
#for i in range(0, len(acc_per_fold_640_ft)):
#  print('------------------------------------------------------------------------')
#  print(f'> Fold {i+1} - Loss: {loss_per_fold_640_ft[i]} - Accuracy: {acc_per_fold_640_ft[i]}% - Precision: {precision_per_fold_640_ft[i]} - Recall: {recall_per_fold_640_ft[i]}')
#print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_640_ft)} (+- {np.std(acc_per_fold_640_ft)})')
print(f'> Loss: {np.mean(loss_per_fold_640_ft)}')
print(f'> F1: {np.mean(f1_fold_640_ft)}(+- {np.std(f1_fold_640_ft)})')
print(f'> Precision: {np.mean(precision_per_fold_640_ft)}(+- {np.std(precision_per_fold_640_ft)})')
print(f'> Recall: {np.mean(recall_per_fold_640_ft)}(+- {np.std(recall_per_fold_640_ft)})')
print('------------------------------------------------------------------------') 

# == Provide average scores ==
print('-------------------------------320-----------------------------------------')
#print('Score per fold')
#for i in range(0, len(acc_per_fold_320_ft)):
#  print('------------------------------------------------------------------------')
#  print(f'> Fold {i+1} - Loss: {loss_per_fold_320_ft[i]} - Accuracy: {acc_per_fold_320_ft[i]}% - Precision: {precision_per_fold_320_ft[i]} - Recall: {recall_per_fold_320_ft[i]}')
#print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_320_ft)} (+- {np.std(acc_per_fold_320_ft)})')
print(f'> Loss: {np.mean(loss_per_fold_320_ft)}')
print(f'> F1: {np.mean(f1_fold_320_ft)}(+- {np.std(f1_fold_320_ft)})')
print(f'> Precision: {np.mean(precision_per_fold_320_ft)}(+- {np.std(precision_per_fold_320_ft)})')
print(f'> Recall: {np.mean(recall_per_fold_320_ft)}(+- {np.std(recall_per_fold_320_ft)})')
print('------------------------------------------------------------------------') 

# == Provide average scores ==
print('-------------------------------160-----------------------------------------')
#print('Score per fold')
#for i in range(0, len(acc_per_fold_160_ft)):
#  print('------------------------------------------------------------------------')
#  print(f'> Fold {i+1} - Loss: {loss_per_fold_160_ft[i]} - Accuracy: {acc_per_fold_160_ft[i]}% - Precision: {precision_per_fold_160_ft[i]} - Recall: {recall_per_fold_160_ft[i]}')
#print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_160_ft)} (+- {np.std(acc_per_fold_160_ft)})')
print(f'> Loss: {np.mean(loss_per_fold_160_ft)}')
print(f'> F1: {np.mean(f1_fold_160_ft)}(+- {np.std(f1_fold_160_ft)})')
print(f'> Precision: {np.mean(precision_per_fold_160_ft)}(+- {np.std(precision_per_fold_160_ft)})')
print(f'> Recall: {np.mean(recall_per_fold_160_ft)}(+- {np.std(recall_per_fold_160_ft)})')
print('------------------------------------------------------------------------') 

# == Provide average scores ==
print('-------------------------------80-----------------------------------------')
#print('Score per fold')
#for i in range(0, len(acc_per_fold_80_ft)):
#  print('------------------------------------------------------------------------')
#  print(f'> Fold {i+1} - Loss: {loss_per_fold_80_ft[i]} - Accuracy: {acc_per_fold_80_ft[i]}% - Precision: {precision_per_fold_80_ft[i]} - Recall: {recall_per_fold_80_ft[i]}')
#print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_80_ft)} (+- {np.std(acc_per_fold_80_ft)})')
print(f'> Loss: {np.mean(loss_per_fold_80_ft)}')
print(f'> F1: {np.mean(f1_fold_80_ft)}(+- {np.std(f1_fold_80_ft)})')
print(f'> Precision: {np.mean(precision_per_fold_80_ft)}(+- {np.std(precision_per_fold_80_ft)})')
print(f'> Recall: {np.mean(recall_per_fold_80_ft)}(+- {np.std(recall_per_fold_80_ft)})')
print('------------------------------------------------------------------------') 

# == Provide average scores ==
print('-------------------------------40-----------------------------------------')
#print('Score per fold')
#for i in range(0, len(acc_per_fold_40_ft)):
#  print('------------------------------------------------------------------------')
#  print(f'> Fold {i+1} - Loss: {loss_per_fold_40_ft[i]} - Accuracy: {acc_per_fold_40_ft[i]}% - Precision: {precision_per_fold_40_ft[i]} - Recall: {recall_per_fold_40_ft[i]}')
#print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold_40_ft)} (+- {np.std(acc_per_fold_40_ft)})')
print(f'> Loss: {np.mean(loss_per_fold_40_ft)}')
print(f'> F1: {np.mean(f1_fold_40_ft)}(+- {np.std(f1_fold_40_ft)})')
print(f'> Precision: {np.mean(precision_per_fold_40_ft)}(+- {np.std(precision_per_fold_40_ft)})')
print(f'> Recall: {np.mean(recall_per_fold_40_ft)}(+- {np.std(recall_per_fold_40_ft)})')
print('------------------------------------------------------------------------') 

-------------------------------640-----------------------------------------
Average scores for all folds:
> Accuracy: 79.3750011920929 (+- 4.011702079701311)
> Loss: 0.7985724210739136
> F1: 64.0613702951308(+- 2.781932801261711)
> Precision: 64.47139263638563(+- 2.5304866197913864)
> Recall: 63.662727054920836(+- 3.0675090590827723)
------------------------------------------------------------------------
-------------------------------320-----------------------------------------
Average scores for all folds:
> Accuracy: 81.37499928474426 (+- 4.391042682696033)
> Loss: 0.7129018783569336
> F1: 64.46158902060483(+- 3.0448723666077147)
> Precision: 64.78681159025422(+- 3.782198656690332)
> Recall: 64.16410614788825(+- 2.426554786882352)
------------------------------------------------------------------------
-------------------------------160-----------------------------------------
Average scores for all folds:
> Accuracy: 80.625 (+- 2.931508833269715)
> Loss: 0.7058194041252136
> F1: 6

In [11]:
#Predictions of the naoAnotados comments
input_predictions_naoAnotados = df_naoAnotados['review_text']
label_predictions_naoAnotados= df_naoAnotados['label']
predictions_naoAnotados_dataset = tf.data.Dataset.from_generator(
    transform_to_dataset(input_predictions_naoAnotados, label_predictions_naoAnotados),
    output_types=({"word_ids": tf.int64,
                  "masks": tf.int64, "type_ids": tf.int64}, tf.int64),
    output_shapes=({"word_ids": (MAX_SEQ_LENGTH),
                    "masks": (MAX_SEQ_LENGTH), "type_ids": (MAX_SEQ_LENGTH)}, ()))
predictions_naoAnotados_dataset = predictions_naoAnotados_dataset.batch(32)

predictions_naoAnotados = model.predict(predictions_naoAnotados_dataset)

df_naoAnotados["predict"]= predictions_naoAnotados.argmax(1) 

print('predictions_naoAnotados:', predictions_naoAnotados.shape)
print(df_naoAnotados.predict.value_counts())
print(df_naoAnotados.label.value_counts())

predictions_naoAnotados: (121496, 3)
2    87655
0    29870
1     3971
Name: predict, dtype: int64
2    73514
0    32998
1    14984
Name: label, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
